ROI Pooling与ROI Align
===

ROI Pooling与ROI Align都是用于两阶段目标识别中，将不同尺寸的特征图，量化为统一大小的特征图的过程

# 1. ROI Pooling
![images](Images/03/02/03_05_001.png)

在常见的两级检测框架（比如Fast-RCNN，Faster-RCNN，RFCN）中，ROI Pooling的作用是根据预选框的位置坐标在特征图中将相应区域池化为固定尺寸的特征图，以便进行后续的分类和包围框回归操作。

我们使用上图来具体分析一下。输入是一张$800 \times 800$的图片，图片上有一个$665 \times 665$的包围框，内部是一只狗。图片经过主干网络提取特征后，特征缩放步长为32，因此图像和包围框的边长都是原来的$\frac{1}{32}$。所以图片边长变为$\frac{800}{32}=25$，包围框边长变为$\frac{665}{32} \approx 20.78$，带有小数，ROI Pooling直接将它量化成$20$。接下来需要把框内的特征池化到$7 \times 7$的大小(为了统一放入全连接层).于是每个小个子的边长就是$\frac{20}{7} \approx 2.86$。于是ROI Pooling又把它量化为2.显然经过两次量化，候选区域已经出现了较为明显的偏差(图中的绿色部分)。更重要的是，该层特征图上0.1个像素的偏差，缩放到原图就是3.2个像素，那么0.8的偏差，在原图上就是接近30个像素点的差别。

# 2.ROI Align
![images](Images/03/02/03_05_002.png)

为了解决ROI Pooling的上述缺点，作者提出了ROI Align这一改进的方法如上图。ROI Align的思路很简单：取消量化操作，使用双线性内插的方法获得坐标为浮点数的像素点上的图像数值,从而将整个特征聚集过程转化为一个连续的操作。

![images](Images/03/02/03_05_003.png)

值得注意的是，在具体的算法操作上，ROI Align并不是简单地补充出候选区域边界上的坐标点，然后将这些坐标点进行池化，而是重新设计了一套比较优雅的流程
- 遍历每一个候选区域，保持浮点数边界不做量化。
- 将候选区域分割成$k \times k$个单元，每个单元的边界也不做量化。
- 在每个单元中计算固定四个坐标位置，用双线性内插的方法计算出这四个位置的值，然后进行最大池化操作。

- 输入是一张$800 \times 800$的图片，图片上有一个$665 \times 665$的包围框，内部是一只狗。图片经过主干网络提取特征后，特征缩放步长为32，因此图像和包围框的边长都是原来的$\frac{1}{32}$。所以图片边长变为$\frac{800}{32}=25$
- 包围框边长变为$\frac{665}{32} \approx 20.78$，这里不进行量化操作
- 接下来需要把框内的特征池化到$7 \times 7$的大小(为了统一放入全连接层).于是每个小个子的边长就是$\frac{20.78}{7} \approx 2.97$，这里也不进行量化

接下来就是最关键的部分，假定采样点数为4，也就是说对于每一个$2.97 \times 2.97$的小区域，平分成四份，每一份取其中心点的位置，而中心点位置的像素，采用双插值法进行计算，这样就会得到四个点的像素值，如下图
![images](Images/03/02/03_05_004.png)
上图中红色叉叉的像素值就是通过双线性插值计算得到的，最后去这四个像素中最大的值作为$2.97 \times 2.97$这个分区域的像素值，一次类推，同样是$7 \times 7$个小区域就有$7 \times 7$个像素值，组成了$7 \times 7$大小的feature map

# 3.ROI Align的反向传播

常规的ROI Pooling的反向传播公式如下
$$\frac{\partial{L}}{\partial{x_i}}=\sum_r\sum_j[i=i*(r,j)]\frac{\partial{L}}{\partial{y_{rj}}}$$
这里$x_i$代表池化前特征图上的像素点；$y_{rj}$代表池化后的第r个候选区域的第j个点；$i*(r,j)$代表点$y_{rj}$像素值的来源(最大池化的时候选出的最大像素值点的坐标)。由上式可以看出，只有当池化后某一个点的像素值在池化过程中采用了当前点Xi的像素值（即满足$i*(r,j)$），才在$x_i$处回传梯度。类比于ROIPooling，ROIAlign的反向传播需要作出稍许修改：首先，在ROIAlign中，$x_i*(r,j)$是一个浮点数的坐标位置(前向传播时计算出来的采样点)，在池化前的特征图中，每一个与$x_i*(r,j)$横纵坐标均小于1的点都应该接受与此对应的点$y_{rj}$回传的梯度，故ROI Align的反向传播公式如下:
$$\frac{\partial{L}}{\partial{x_i}}=\sum_r\sum_j[d(i, i*(r,j)) < 1](1-\Delta{h})(1-\Delta{w})\frac{\partial{L}}{\partial{y_{rj}}}$$
上式中，$d(.)$表示两点之间的距离，$\Delta{h}$和$\Delta{w}$表示$x_i$与$x_i*(r,j)$横纵坐标的差值，这里作为双线性内插的系数乘在原始的梯度上。